**Exercise 1**

In [20]:
!pip install -q pyomo

\begin{align}
\min \ c_1 x_1 + c_2 x_2 + \ldots + c_N x_N, \\
\; x_1 +  x_2 + \ldots + x_N & \leq b_1, \\
w_1 x_1 + w_2 x_2 + \ldots + w_N x_N  & = b_2, \\
x_i & \leq u_i, \quad i=1,\ldots,N, \\
x_i & \geq l_i, \quad i=1,\ldots,N.  
\end{align}
 let us consider N = 12,then the given problem will looks like
$\min \ -8.1x_1 + 10.15x_2 + 30.5x_3 + 50.05x_4+ 0.05x_5 + 80.5x_6 -0.25x_7 -31.02x_8 + 50.65x_9 + 0.725x_{10} -0.8x_{11} + 100.6x_{12} \\ \text{ s.t. }  \\ 
x_1 + x_2 + x_3 + x_4 + x_5 + x_6 + x_7 + x_8 + x_9 + x_{10} + x_{11} + x_{12}\leq 161.2, \\  1.6x_2 - 2.02x_3 + 1.01x_5 - 2.005x_6 + 1.39x_8 + x_9 -1.214 x_{10} - 5.32x_{12} = 54.6, \\ -\infty \leq x_1 \leq4,\ 1\leq x_2 \leq 3, 0 \leq x_3\leq \infty, 0\leq x_4 \leq2 , 7\leq x_5 \leq 10,\\ 0\leq x_6\leq \infty, -\infty \leq x_7\leq 13, -\infty \leq x_8 \leq 20, 1\leq x_9 \leq \infty,\\ 1\leq x_{10} \leq 21, -5\leq x_{11} \leq 5, 2\leq x_{12} \leq 60;    \\ l = [-\infty, 1, 0, 0, 7, 0, -\infty, -\infty , 1, 1, -5, 2  ]; \\
 u = [4, 3, +\infty, 2, 10, +\infty, 13, 20, +\infty, 21, 5, 60] $

 

  

In [21]:
from pyomo.environ import *

In [22]:
import numpy as np

In [23]:
#create a model
model = ConcreteModel()

In [24]:
# we can set certain values using python variables
N = 12 #we declare a Python variable N denoting the number of decision variables  

In [25]:
# coefficients for objective function can be provided using numpy arrays
obj_coef_c = np.array([-8.1, 10.15, 30.5, 50.05, 0.05, 80.5, -0.25, -31.02, 50.65, 0.725, -0.8, 100.6]) #We declare a numpy array by passing the values in a list

In [26]:
# We use more numpy arrays for providing constraint coefficients
constr1_coef_p = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
constr2_coef_q = np.array([0, 1.6, -2.02, 0, 1.01, -2.005, 0, 1.39, 1, -1.214, 0, -5.32])

#declare the RHS of the constraints as variables
constr1_rhs_b1 = 161.2
constr2_rhs_b2 = 54.6

In [27]:
#The following numpy arrays will be useful for setting the lower bound and upper bounds for the variables
lower_bound = np.array([-np.inf, 1, 0, 0, 7, 0, -np.inf, -np.inf , 1, 1, -5, 2 ])
upper_bound = np.array([4, 3, np.inf, 2, 10, np.inf, 13, 20, np.inf, 21, 5, 60])

In [28]:
# set of column indices: since we have N variables, we can take column indices to be the list [0,1,2,...,N-1]
col_indices = np.arange(N)
print(col_indices)

[ 0  1  2  3  4  5  6  7  8  9 10 11]


In [29]:
#declare the decision variables in the model
model.x = Var(col_indices)

In [30]:
# create a ConstraintList to hold multiple constraints
model.constraints = ConstraintList()

In [31]:
#add model constraints one by one to the list 
#first add the constraint  x1+ x2+...+ xN <= b1 
model.constraints.add(sum(constr1_coef_p[j]*model.x[j] for j in col_indices) <= constr1_rhs_b1)

In [32]:
#next add the constraint w1 x1 + w2 x2 + ...+ wN xN == b2 
model.constraints.add(sum(constr2_coef_q[j]*model.x[j] for j in col_indices) == constr2_rhs_b2)

In [33]:
#add the upper bound and lower bounds for the variables
for j in col_indices:
  model.x[j].setlb(lower_bound[j])
  model.x[j].setub(upper_bound[j])

In [34]:
# add the model objective
model.objective = Objective(expr = sum(obj_coef_c[j]*model.x[j] for j in col_indices), sense=minimize)

In [35]:
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {1, 2}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}

1 Var Declarations
    x : Size=12, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :  -inf :  None :   4.0 : False :  True :  Reals
          1 :   1.0 :  None :   3.0 : False :  True :  Reals
          2 :   0.0 :  None :   inf : False :  True :  Reals
          3 :   0.0 :  None :   2.0 : False :  True :  Reals
          4 :   7.0 :  None :  10.0 : False :  True :  Reals
          5 :   0.0 :  None :   inf : False :  True :  Reals
          6 :  -inf :  None :  13.0 : False :  True :  Reals
          7 :  -inf :  None :  20.0 : False :  True :  Reals
          8 :   1.0 :  None :   inf : False :  True :

In [36]:
!apt-get install -y -qq glpk-utils

In [37]:
SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 775.965100000001
  Upper bound: 775.965100000001
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 13
  Number of nonzeros: 21
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.01157999038696289
# ----------------------------------------------------------
#   Solution Information
# -------------

In [38]:
# display solution
print('\nCost = Rs.', model.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i+1,'] = ', model.x[i].value)

print('\nConstraints')
model.constraints.display()


Cost = Rs. 775.9651000000001

Decision Variables
x[ 1 ] =  4.0
x[ 2 ] =  3.0
x[ 3 ] =  0.0
x[ 4 ] =  0.0
x[ 5 ] =  10.0
x[ 6 ] =  0.0
x[ 7 ] =  13.0
x[ 8 ] =  20.0
x[ 9 ] =  23.754
x[ 10 ] =  1.0
x[ 11 ] =  5.0
x[ 12 ] =  2.0

Constraints
constraints : Size=2
    Key : Lower : Body               : Upper
      1 :  None :             81.754 : 161.2
      2 :  54.6 : 54.599999999999994 :  54.6
